HDB Resale Transaction bot - by M. W. Ho (c) 2022


In [9]:
!pip install python-telegram-bot

import pandas as pd
import requests
import telegram

from tabulate import tabulate
from pandas.plotting import table 

bot = telegram.Bot(token='5218309496:AAHV2gIFLgNSK37D98kvFScBiLMxorHotJU')
print("\n",bot.get_me(),"\n")

from telegram.ext import Updater
updater = Updater(token='5218309496:AAHV2gIFLgNSK37D98kvFScBiLMxorHotJU', use_context=True)
dispatcher = updater.dispatcher

import logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                     level=logging.INFO)

ModuleNotFoundError: No module named 'tabulate'

In [7]:
###
from telegram import Update
from telegram.ext import CallbackContext

def start(update: Update, context: CallbackContext):
    context.bot.send_message(chat_id=update.effective_chat.id, text="HDBRT Bot - Get latest HDB Resale Transactions.\nType / for commands.")

def last10(update: Update, context: CallbackContext):
    q=' '.join(context.args)
    url = "https://data.gov.sg/api/action/datastore_search?resource_id=f1765b54-a209-4718-8d38-a39237f502b3&fields=month,block,street_name,flat_type,flat_model,resale_price,floor_area_sqm,lease_commence_date,storey_range&sort=month%20desc&limit=10&q="+q
    response=requests.get(url)
    dataset=response.json()
    df = pd.json_normalize(dataset['result'], "records")
    df = df.sort_values(by='block').set_index('block').reindex(columns=['street_name','flat_type','resale_price','lease_commence_date','storey_range','floor_area_sqm','flat_model','month'])
    df.replace(['2 ROOM', '3 ROOM', '4 ROOM', '5 ROOM', 'New Generation', 'Standard'],['2Rm','3Rm','4Rm','5Rm','NG','Std'], inplace=True) 
    context.bot.send_message(chat_id=update.effective_chat.id, text=tabulate(df, headers='keys', tablefmt='simple', numalign='left'))
    
def last20(update: Update, context: CallbackContext):
    q=' '.join(context.args)
    url = "https://data.gov.sg/api/action/datastore_search?resource_id=f1765b54-a209-4718-8d38-a39237f502b3&fields=month,block,street_name,flat_type,flat_model,resale_price,floor_area_sqm,lease_commence_date,storey_range&sort=month%20desc&limit=20&q="+q
    response=requests.get(url)
    dataset=response.json()
    df = pd.json_normalize(dataset['result'], "records")
    df = df.sort_values(by='block').set_index('block').reindex(columns=['street_name','flat_type','resale_price','lease_commence_date','storey_range','floor_area_sqm','flat_model','month'])
    df.replace(['2 ROOM', '3 ROOM', '4 ROOM', '5 ROOM', 'New Generation', 'Standard'],['2Rm','3Rm','4Rm','5Rm','NG','Std'], inplace=True) 
    context.bot.send_message(chat_id=update.effective_chat.id, text=tabulate(df, headers='keys', tablefmt='simple', numalign='left'))

def last30(update: Update, context: CallbackContext):
    q=' '.join(context.args)
    url = "https://data.gov.sg/api/action/datastore_search?resource_id=f1765b54-a209-4718-8d38-a39237f502b3&fields=month,block,street_name,flat_type,flat_model,resale_price,floor_area_sqm,lease_commence_date,storey_range&sort=month%20desc&limit=30&q="+q
    response=requests.get(url)
    dataset=response.json()
    df = pd.json_normalize(dataset['result'], "records")
    df = df.sort_values(by='block').set_index('block').reindex(columns=['street_name','flat_type','resale_price','lease_commence_date','storey_range','floor_area_sqm','flat_model','month'])
    df.replace(['2 ROOM', '3 ROOM', '4 ROOM', '5 ROOM', 'New Generation', 'Standard'],['2Rm','3Rm','4Rm','5Rm','NG','Std'], inplace=True) 
    context.bot.send_message(chat_id=update.effective_chat.id, text=tabulate(df, headers='keys', tablefmt='simple', numalign='left'))
    

In [ ]:
### for testing API and response only
url = "https://data.gov.sg/api/action/datastore_search?resource_id=f1765b54-a209-4718-8d38-a39237f502b3&fields=month,block,street_name,flat_type,flat_model,resale_price,floor_area_sqm,lease_commence_date,storey_range&sort=month%20desc&limit=20&q="
response=requests.get(url)
dataset=response.json()
df = pd.json_normalize(dataset['result'], "records")  
df = df.sort_values(by='block').set_index('block').reindex(columns=['street_name','flat_type','resale_price','lease_commence_date','storey_range','floor_area_sqm','flat_model','month'])
df.replace(['2 ROOM', '3 ROOM', '4 ROOM', '5 ROOM', 'New Generation', 'Standard'],['2Rm','3Rm','4Rm','5Rm','NG','Std'], inplace=True) 
print(df)

import matplotlib.pyplot as plt

ax = plt.subplot(frame_on=False) # no visible frame
ax.xaxis.set_visible(False)  # hide the x axis
ax.yaxis.set_visible(False)  # hide the y axis
table(ax, df)  # where df is your data frame
plt.rcParams["figure.figsize"] = (22, 1)
plt.savefig('mytable.png')


In [ ]:
###
from telegram.ext import CommandHandler

start_handler = CommandHandler('start', start)
dispatcher.add_handler(start_handler)

last10_handler = CommandHandler('last10', last10)
dispatcher.add_handler(last10_handler)

last20_handler = CommandHandler('last20', last20)
dispatcher.add_handler(last20_handler)

last30_handler = CommandHandler('last30', last30)
dispatcher.add_handler(last30_handler)

bot.send_message(text='Launched!', chat_id=1349303768)

#####
updater.start_polling()

